# PyOpenCL

In [3]:
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
from pyopencl.scan import GenericScanKernel
import numpy as np

In [4]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

### Enumerate

Scan of true/false array  

\[ 1, 0, 1, 1, 0, 0 \]  
\[ 0, 1, 1, 2, 3, 3 \]  

Number of true elements to the left

In [9]:
x = np.array([1, 0, 1, 1, 0, 0], dtype=np.int32)
x_g = cl.array.to_device(queue, x)
out = cl.array.zeros_like(x_g)

In [26]:
sknl = GenericScanKernel(
        ctx, np.int32,
        arguments="__global int *ary, __global int *out",
        input_expr="ary[i]",
        scan_expr="a+b", neutral="0",
        output_statement="out[i+1] = item;")

In [27]:
sknl(x_g, out)

In [28]:
print(out)

[0 1 1 2 3 3]


### Implementing conditional copy using enumerate

In [29]:
data = np.array([0, 1, 2, 3, 4, 5], dtype=np.int32)
data = cl.array.to_device(queue, data)

In [30]:
out = cl.array.zeros_like(x_g)

In [31]:
sknl(x_g, out)
print(out)

[0 1 1 2 3 3]


In [32]:
from pyopencl.elementwise import ElementwiseKernel

In [33]:
knl = ElementwiseKernel(ctx, "int* tags, int* indices, int* data, int* res",
                       """
                       if(tags[i] == 1)
                           res[indices[i]] = data[i];
                       """)

In [36]:
res = cl.array.zeros_like(data)

knl(x_g, out, data, res)

In [37]:
print(res)

[0 2 3 0 0 0]


## Exercise

Split an array in two parts according to the tags using enumerate

\[ 0, 1, 2, 3, 4, 5 \]  
\[ 1, 0, 1, 1, 0, 0 \]  

Result:  
\[ 0, 2, 3, 1, 4, 5 \]